In [1]:
import math
import random
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
def set_thresholds(G, threshold):
    """Set the same activation threshold for all nodes in the graph."""
    for node in G.nodes():
        G.nodes[node]['T'] = threshold
    return G

def reset_states(G):
    """Reset the activation states and edge utilization for all nodes and edges in the graph."""
    for node in G.nodes():
        G.nodes[node]['Trigger'] = None
        G.nodes[node]['Active'] = False
        G.nodes[node]['Active_Neighbor_count'] = -1
        G.nodes[node]['Activation_time'] = -1
    for edge in G.edges():
        G.edges[edge]['Utilization'] = 'N'
    return G

def make_spread(G, q, max_steps=math.inf):
    """Simulate the spread of contagion on the graph until either full spread or the step limit is reached."""
    time_step = 0

    while calc_spreading_density(G) < 1 and time_step < max_steps:
        time_step += 1
        
        # Count active neighbors for inactive nodes
        for node in G.nodes():
            if not G.nodes[node]['Active']:
                active_neighbors = sum(
                    1 for neighbor in G.neighbors(node) 
                    if G.nodes[neighbor]['Active'] and G.nodes[neighbor]['Activation_time'] < time_step
                )
                G.nodes[node]['Active_Neighbor_count'] = active_neighbors
        
        # Trigger nodes based on neighbor count exceeding the threshold
        for node in G.nodes():
            if G.nodes[node]['Active_Neighbor_count'] >= G.nodes[node]['T'] and not G.nodes[node]['Active']:
                G.nodes[node]['Active'] = True
                G.nodes[node]['Trigger'] = 'T'
                G.nodes[node]['Activation_time'] = time_step
                
                # Mark edges as utilized if neighbors activated earlier
                for neighbor in G.neighbors(node):
                    if G.nodes[neighbor]['Active'] and G.nodes[neighbor]['Activation_time'] < time_step:
                        G.edges[neighbor, node]['Utilization'] = 'T'

        # Random activation of neighbors based on probability q
        for node in G.nodes():
            if G.nodes[node]['Active'] and G.nodes[node]['Activation_time'] < time_step:
                for neighbor in G.neighbors(node):
                    if not G.nodes[neighbor]['Active'] and random.random() < q:
                        G.nodes[neighbor]['Active'] = True
                        G.nodes[neighbor]['Trigger'] = 'Q'
                        G.nodes[neighbor]['Activation_time'] = time_step
                        G.edges[node, neighbor]['Utilization'] = 'Q'

    return G, time_step

def calc_spreading_density(G):
    """Calculate the proportion of active nodes in the graph."""
    active_count = sum(1 for node in G.nodes() if G.nodes[node]['Active'])
    return active_count / len(G.nodes())

def calc_tie_ranges(G):
    """Calculate the tie range (second shortest path length) for each edge in the graph."""
    for edge in G.edges():
        G_copy = G.copy()
        G_copy.remove_edge(*edge)
        G.edges[edge]['Tie_range'] = nx.shortest_path_length(G_copy, source=edge[0], target=edge[1])
    return G

def watts_strogatz_rewired(n, k, eta):
    """Generate a Watts-Strogatz graph and adjust rewiring probability based on desired rewiring count."""
    total_edges = n * k // 2  # Total number of edges in the regular ring lattice
    rewiring_prob = eta / total_edges  # Rewiring probability for η rewirings
    return nx.watts_strogatz_graph(n, k, rewiring_prob)


In [ ]:
# Define parameters
p_values = [0.05, 0, 0.05]
T_values = [math.inf, 2, 2]
NODE_SIZE = 30
titles1 = ['a', 'b', 'c']
titles2 = ['d', 'e', 'f']

# Create a subplot with 2 rows and 3 columns
fig, axs = plt.subplots(2, 3, figsize=(12, 10))

# Set aspect ratio to 'equal' for all axes
for ax in axs.flatten():
    ax.set_aspect('equal')

# Create a Watts-Strogatz graph
G = nx.watts_strogatz_graph(100, 4, 0)
seed = 11

# Plot G in a circular layout and move every second node to the inside of the circle
pos = nx.circular_layout(G)
for j in range(0, len(G.nodes()), 2):
    pos[j] *= 0.6

# Loop through each p_value to simulate the spread and plot the results
for i in range(len(p_values)):
    random.seed(1)  # Ensures reproducibility

    # Apply thresholds and reset states
    G = set_thresholds(G, T_values[i])
    G = reset_states(G)

    # Manually activate the seed node and its neighbors
    G.nodes[seed]['Active'] = True
    G.nodes[seed]['Activation_time'] = 0
    for neighbor in G.neighbors(seed):
        G.nodes[neighbor]['Active'] = True
        G.nodes[neighbor]['Activation_time'] = 0

    # Simulate the spread
    G, num_steps = make_spread(G, p_values[i], 29)

    # Plot the graph on the top row
    nx.draw(G, pos, ax=axs[0, i], with_labels=False, node_color='grey', node_size=NODE_SIZE, width=0.5, alpha=0.5)
    nx.draw_networkx_nodes(G, pos, ax=axs[0, i], nodelist=[n for n in G.nodes() if G.nodes[n]['Active']],
                           node_color='red', node_size=NODE_SIZE, label='Seed nodes')
    nx.draw_networkx_nodes(G, pos, ax=axs[0, i], nodelist=[n for n in G.nodes() if G.nodes[n]['Active'] and G.nodes[n]['Trigger'] == 'T'],
                           node_color='blue', node_size=NODE_SIZE, label='Deterministically activated nodes')
    nx.draw_networkx_nodes(G, pos, ax=axs[0, i], nodelist=[n for n in G.nodes() if G.nodes[n]['Active'] and G.nodes[n]['Trigger'] == 'Q'],
                           node_color='orange', node_size=NODE_SIZE, label='Noisy Subthreshold Activated Nodes')
    
    axs[0, i].set_title(titles1[i], fontweight='bold')

# Add an additional edge to the graph
G.add_edge(10, 60)

# Repeat the same process for the second row of subplots
for i in range(len(p_values)):
    random.seed(1)  # Ensures reproducibility

    # Apply thresholds and reset states
    G = set_thresholds(G, T_values[i])
    G = reset_states(G)

    # Manually activate the seed node and its neighbors
    G.nodes[seed]['Active'] = True
    G.nodes[seed]['Activation_time'] = 0
    for neighbor in G.neighbors(seed):
        G.nodes[neighbor]['Active'] = True
        G.nodes[neighbor]['Activation_time'] = 0

    # Simulate the spread
    G, num_steps = make_spread(G, p_values[i], 29)

    # Plot the graph on the bottom row
    nx.draw(G, pos, ax=axs[1, i], with_labels=False, node_color='grey', node_size=NODE_SIZE, width=0.5, alpha=0.5)
    nx.draw_networkx_nodes(G, pos, ax=axs[1, i], nodelist=[n for n in G.nodes() if G.nodes[n]['Active']],
                           node_color='red', node_size=NODE_SIZE, label='Seed nodes')
    nx.draw_networkx_nodes(G, pos, ax=axs[1, i], nodelist=[n for n in G.nodes() if G.nodes[n]['Active'] and G.nodes[n]['Trigger'] == 'T'],
                           node_color='blue', node_size=NODE_SIZE, label='Deterministically activated nodes')
    nx.draw_networkx_nodes(G, pos, ax=axs[1, i], nodelist=[n for n in G.nodes() if G.nodes[n]['Active'] and G.nodes[n]['Trigger'] == 'Q'],
                           node_color='orange', node_size=NODE_SIZE, label='Noisy Subthreshold Activated Nodes')
    
    axs[1, i].set_title(titles2[i], fontweight='bold')

# Optimize the layout for better readability
plt.tight_layout()

# Add the legend below the subplots
plt.legend(loc='lower center', bbox_to_anchor=(0, -0.05), ncol=3)

# Add a text box at the bottom with some details
plt.text(0.5, 0.51, 'All simulations carried out over 29 timesteps', fontsize=12, color='black',
         bbox=dict(facecolor='none', edgecolor='black', boxstyle='round,pad=0.5'),
         ha='center', transform=plt.gcf().transFigure)

# Save the plot with high resolution
plt.savefig('plot_comparison.png', dpi=300)


In [ ]:
# Initialize list for value pairs
value_pairs = []
n = 500  # Number of nodes
k = 4    # Number of neighbors

# Loop through eta values and q values
for eta in tqdm(range(int(np.sqrt(n)))):
    for q in np.arange(0, 0.2, 0.01):
        for repeat in range(10):
            # Create rewired Watts-Strogatz graph
            G = watts_strogatz_rewired(n, k, eta)

            # Calculate tie ranges and reset node/edge states
            G = calc_tie_ranges(G)
            G = set_thresholds(G, 2)
            G = reset_states(G)

            # Manually activate the seed node and its neighbors
            G.nodes[seed]['Active'] = True
            G.nodes[seed]['Activation_time'] = 0
            for neighbor in G.neighbors(seed):
                G.nodes[neighbor]['Active'] = True
                G.nodes[neighbor]['Activation_time'] = 0

            # Simulate the spread
            G, num_steps = make_spread(G, q, max_steps=G.number_of_nodes())

            # Collect tie range and utilization for each edge
            for edge in G.edges():
                value_pairs.append((G.edges[edge]['Tie_range'], G.edges[edge]['Utilization']))

# Create a DataFrame from the list of tuples
df = pd.DataFrame(value_pairs, columns=['Tie range', 'Utilization'])

# Filter out rows where Utilization is 'N'
df = df[df['Utilization'] != 'N']

# Plot a stacked histogram of the tie ranges with Utilization as the hue
plt.figure(figsize=(15, 5))
sns.histplot(data=df, 
             x='Tie range', 
             hue='Utilization', 
             multiple='dodge', 
             palette=['blue', 'orange'], 
             edgecolor=".3",
             linewidth=0.5,
             binwidth=1)  # Adjust binwidth for wider bars

# Set y-axis to a semi-logarithmic scale
plt.yscale('log')

# Update legend labels
plt.legend(labels=['Noisy Subthreshold Activation Edge Utilization', 'Deterministic Threshold Activation Edge Utilization'])

# Save the plot with high resolution
plt.savefig('histogram.png', dpi=300)